Introduction
============

This is interactive notebook regarding "Introduction to path planning". (Author: Björn Hein)

License is based on Creative Commons: Attribution-NonCommercial 4.0 International (CC BY-NC 4.0) (pls. check: http://creativecommons.org/licenses/by-nc/4.0/)

This notebook imports all discussed algorithms and does a comparison

**Attention**: This worksheet has external dependencies 

* to the files "InnoKonz2dEnv.py" or "InnoKonzShapelyEnv.py". These files contain simple 2D environments based on numpy arrays or the shapely library with an interface for the use as collision checking in the algorithms.
* to all modules like "**Algorithm**.py", which contains the corresponding algorithms 

Important links are:

* General Info: http://www.python.org
* Python tutorial http://www.python.org/doc/tut/
* NetworkX http://networkx.github.io/
* NumPy and SciPy Guide http://docs.scipy.org/
* Matplotlib gallery http://matplotlib.sourceforge.net/gallery.html


Remember that:

* you have to press ctrl-return or shift-return, to execute the code in the code sections, only then the variables are "generated" and can be used
* you can execute the whole notebook by Cell->runAll

Adding all planners
===========


In [21]:
import sys
import HelperClass
sys.path.append("templates")
%load_ext autoreload
%autoreload 2
import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
from IPPerfMonitor import IPPerfMonitor

from BasicPRM import IPVISBasicPRM 
from BasicPRM import IPBasicPRM_Roundtrip

from VisibilityPRM import IPVisibilityPRM_Roundtrip
from VisibilityPRM import IPVISVisibilityPRM

from VisibilityPRM import CC_IPVisibilityPRM_Roundtrip

from LazyPRM import IPLazyPRM_Roundtrip
from LazyPRM import IPVISLazyPRM

from IPython.utils import io



Set-up of the test scenario and the configuration for all planner
===================================

Following a procedure to compare all discussed planners are shown:

1. Configuration for every planner is defined
2. The configuration and the planner is stored in the variable setup, a Python dict()
3. the variable setup is then used to uniformly call the planning calls


In [23]:
plannerFactory = dict()

basicConfig = dict()
basicConfig["radius"] = 3
basicConfig["numNodes"] = 200
plannerFactory["basePRM"] = [IPBasicPRM_Roundtrip.BasicPRM, basicConfig, IPVISBasicPRM.basicPRMVisualize]


visbilityConfig = dict()
visbilityConfig["ntry"] = 300
plannerFactory["visibilityPRM"] = [IPVisibilityPRM_Roundtrip.VisPRM, visbilityConfig, IPVISVisibilityPRM.visibilityPRMVisualize ]


visbilityConfig = dict()
visbilityConfig["ntry"] = 300
plannerFactory["visibilityPRMOptimized"] = [CC_IPVisibilityPRM_Roundtrip.VisPRM, visbilityConfig, IPVISVisibilityPRM.visibilityPRMVisualize ]


lazyConfig = dict()
lazyConfig["initialRoadmapSize"] = 10
lazyConfig["updateRoadmapSize"]  = 5
lazyConfig["kNearest"] = 8
plannerFactory["lazyPRM"] = [IPLazyPRM_Roundtrip.LazyPRM, lazyConfig, IPVISLazyPRM.lazyPRMVisualize]

In [24]:
class ResultCollection (object):
    
    def __init__(self, plannerFactoryName, planner, benchmark, solution, perfDataFrame):
        self.plannerFactoryName = plannerFactoryName
        self.planner = planner
        self.benchmark = benchmark
        self.solution = solution
        self.perfDataFrame = perfDataFrame

In [25]:
import IPTestSuite
import importlib

In [26]:
importlib.reload(IPTestSuite)

<module 'IPTestSuite' from 'c:\\Users\\andre\\OneDrive - Hochschule Karlsruhe\\HS Karlsruhe\\1. Semester\\Roboterprogrammierung\\Projekt\\Git\\RKIM23RobotProgramDS\\02_eigeneNotebooks\\IPTestSuite.py'>

In [27]:
fullBenchList = IPTestSuite.benchList

for benchmark in fullBenchList:
    print(benchmark.name)

Trap
Bottleneck
Fat bottleneck
B_rush
circle od death


In [28]:
# List which save the time
# 1. Zeit
# 2. Nodesanzahl
# 3. Erfolgreich

sum_time_Basic = [[],[],[]]
sum_time_Visibility = [[],[],[]]
sum_time_Visibility_Optimized= [[],[],[]]
sum_time_Lazy = [[],[],[]]

list_of_benchmarks = []

In [29]:
resultList = list()
testList = fullBenchList[:5]


amountOfRepetition = 0
i = 0
while (i< amountOfRepetition):
    testList.extend(fullBenchList[:5])
    i = i + 1
 
print(testList)

for key,producer in list(plannerFactory.items()):
    HelperClass.HelperClass.printInColor(str(key)+" "+ str(producer), 'yellow')

    for benchmark in testList:
        #HelperClass.HelperClass.printInColor("Planning: " + key + " - " + benchmark.name, 'yellow')
        planner = producer[0](benchmark.collisionChecker)
        IPPerfMonitor.clearData()
        

        try:

            print(benchmark.name)

            with io.capture_output() as captured:
                start = time.time()
                resultList.append(ResultCollection(key,
                                            planner, 
                                            benchmark, 
                                            planner.planRoundPath(benchmark.startList,benchmark.interimGoalList,benchmark.goalList,producer[1]),
                                            IPPerfMonitor.dataFrame()),)
                end = time.time()
                duration = end - start
            if (key == "basePRM"):
                # sum_time_Basic[0].append(resultList[0].perfDataFrame.groupby(["name"]).sum(numeric_only=True)["time"].sum())
                sum_time_Basic[0].append(duration)
                                   
                sum_time_Basic[1].append(len(resultList[0].planner.graph.nodes()))
                if (resultList[0].solution != []):
                    sum_time_Basic[2].append(1)
                else:
                    sum_time_Basic[2].append(0)

                
            elif (key == "visibilityPRM"):
                sum_time_Visibility[0].append(duration)
                sum_time_Visibility[1].append(len(resultList[0].planner.graph.nodes()))
                
                if (resultList[0].solution != []):
                    sum_time_Visibility[2].append(1)
                else:
                    sum_time_Visibility[2].append(0)
            
            elif (key == "visibilityPRMOptimized"):
                sum_time_Visibility_Optimized[0].append(duration)
                
                sum_time_Visibility_Optimized[1].append(len(resultList[0].planner.graph.nodes()))
                
                if (resultList[0].solution != []):
                    sum_time_Visibility_Optimized[2].append(1)
                else:
                    sum_time_Visibility_Optimized[2].append(0)


            
            elif (key == "lazyPRM"):
                sum_time_Lazy[0].append(duration)
                sum_time_Lazy[1].append(len(resultList[0].planner.graph.nodes()))
                
                if (resultList[0].solution != []):
                    sum_time_Lazy[2].append(1)
                else:
                    sum_time_Lazy[2].append(0)

            list_of_benchmarks.append(benchmark.name)
            #print(resultList[0].perfDataFrame.groupby(["name"]).sum(numeric_only=True).sum()["time"])
            #print(resultList[0].solution)
            #print("Länge ", len(resultList[0].planner.graph.nodes()))
            resultList = list()
            #print(len(resultList))
                
                
            #print("Habe fertig")



        except Exception as e:
            HelperClass.HelperClass.printInColor("PLANNING ERROR ! PLANNING ERROR ! PLANNING ERROR", 'red')
            HelperClass.HelperClass.printInColor(e, 'red')
            import traceback
            HelperClass.HelperClass.printInColor(traceback.print_exc(), 'red')

            pass

        

[<IPBenchmark.Benchmark object at 0x0000026D693DFC10>, <IPBenchmark.Benchmark object at 0x0000026D693CBD90>, <IPBenchmark.Benchmark object at 0x0000026D693CAAD0>, <IPBenchmark.Benchmark object at 0x0000026D693E3A10>, <IPBenchmark.Benchmark object at 0x0000026D697A3890>]


Trap
1.0100560188293457
name
_getRandomFreePosition           0.006001
_inSameConnectedComponent        0.026258
_learnRoadmapNearestNeighbour    0.270363
_nearestInterim                  0.011015
_nearestNeighbours               0.091521
lineInCollision                  0.158665
planRoundPath                    0.318704
pointInCollision                 0.127529
Name: time, dtype: float64
Bottleneck
1.438047170639038
name
_getRandomFreePosition           0.005994
_inSameConnectedComponent        0.026517
_learnRoadmapNearestNeighbour    0.389558
_nearestInterim                  0.000000
_nearestNeighbours               0.093911
lineInCollision                  0.270136
planRoundPath                    0.400558
pointInCollision                 0.251374
Name: time, dtype: float64
Fat bottleneck
1.0518300533294678
name
_getRandomFreePosition           0.014009
_inSameConnectedComponent        0.036307
_learnRoadmapNearestNeighbour    0.294217
_nearestInterim                  0.001001
_nea

Trap
1.915511131286621
name
_checkConnectableInterims    0.008003
_getRandomFreePosition       0.002507
_learnRoadmap                0.495960
_nearestInterim              0.000000
lineInCollision              0.495456
planRoundPath                0.507962
pointInCollision             0.405622
Name: time, dtype: float64
Bottleneck
1.5446147918701172
name
_checkConnectableInterims    0.006510
_getRandomFreePosition       0.007996
_learnRoadmap                0.397397
_nearestInterim              0.000000
lineInCollision              0.389909
planRoundPath                0.407913
pointInCollision             0.334891
Name: time, dtype: float64
Fat bottleneck
1.3966591358184814
name
_checkConnectableInterims    0.006000
_getRandomFreePosition       0.011997
_learnRoadmap                0.361178
_nearestInterim              0.000000
lineInCollision              0.349180
planRoundPath                0.370179
pointInCollision             0.298125
Name: time, dtype: float64
B_rush
40.005876779

Trap
0.22106647491455078
name
_checkConnectableInterims    0.008006
_getRandomFreePosition       0.000000
_learnRoadmap                0.048512
_nearestInterim              0.000000
lineInCollision              0.056517
planRoundPath                0.060518
pointInCollision             0.047513
Name: time, dtype: float64
Bottleneck
0.18601226806640625
name
_checkConnectableInterims    0.007001
_getRandomFreePosition       0.001002
_learnRoadmap                0.041004
_nearestInterim              0.002001
lineInCollision              0.046002
planRoundPath                0.051006
pointInCollision             0.037996
Name: time, dtype: float64
Fat bottleneck
0.33502197265625
name
_checkConnectableInterims    0.005998
_getRandomFreePosition       0.001000
_learnRoadmap                0.082011
_nearestInterim              0.000000
lineInCollision              0.085010
planRoundPath                0.092007
pointInCollision             0.068996
Name: time, dtype: float64
B_rush
0.894517183

Trap
0.05106687545776367
name
_buildRoadmap                  0.004014
_checkForCollisionAndUpdate    0.010836
_nearestInterim                0.001001
lineInCollision                0.009836
planRoundPath                  0.017865
pointInCollision               0.007515
Name: time, dtype: float64
Bottleneck
0.1100459098815918
name
_buildRoadmap                  0.007018
_checkForCollisionAndUpdate    0.023501
_nearestInterim                0.001003
lineInCollision                0.022501
planRoundPath                  0.035523
pointInCollision               0.020500
Name: time, dtype: float64
Fat bottleneck
0.10627055168151855
name
_buildRoadmap                  0.009052
_checkForCollisionAndUpdate    0.019799
_nearestInterim                0.002996
lineInCollision                0.019799
planRoundPath                  0.040838
pointInCollision               0.013788
Name: time, dtype: float64
B_rush
2.4818854331970215
name
_buildRoadmap                  0.129426
_checkForCollisionAndUp

In [30]:
print(sum_time_Lazy)
import pandas as pd
import os
from openpyxl import load_workbook

result_frames = []

print(len(sum_time_Basic[0]))

for i in range(len(sum_time_Basic[0])):
    
    # DataFrame erstellen
    data = {
        "Benchmark": [list_of_benchmarks[i]],
        "Basic Zeit": [sum_time_Basic[0][i]],
        "Basic Nodes": [sum_time_Basic[1][i]],
        "Basic Erfolgreich": [sum_time_Basic[2][i]],
        "Visibility Zeit ": [sum_time_Visibility[0][i]],
        "Visibility Nodes ": [sum_time_Visibility[1][i]],
        "Visibility Erfolgreich ": [sum_time_Visibility[2][i]],
        "Optimized_Visibility Zeit ": [sum_time_Visibility_Optimized[0][i]],
        "Optimized_Visibility Nodes ": [sum_time_Visibility_Optimized[1][i]],
        "Optimized_Visibility Erfolgreich ": [sum_time_Visibility_Optimized[2][i]],
        "LazyPRM Zeit ": [sum_time_Lazy[0][i]],
        "LazyPRM Nodes ": [sum_time_Lazy[1][i]],
        "LazyPRM Erfolgreich ": [sum_time_Lazy[2][i]],
    }

    df = pd.DataFrame(data)

    # DataFrame zur Liste hinzufügen
    result_frames.append(df)

final_df = pd.concat(result_frames, ignore_index=True)
print(type(final_df))
#df = df.append(pd.DataFrame(neue_daten), ignore_index=True)

excel_dateipfad = 'Auswertung_alle_3PRMs.xlsx'
if (os.path.isfile(excel_dateipfad)):
    print("Datei da")
    dataframe = pd.read_excel(excel_dateipfad)
    print(type(dataframe))
    dataframe = pd.concat([dataframe, final_df], ignore_index=True)
    #dataframe.append(final_df, ignore_index=True)
    


# DataFrame in eine Excel-Datei schreiben
dataframe.to_excel("Auswertung_alle_3PRMs.xlsx", index=False)


print("Habe fertig")




[[0.05106687545776367, 0.1100459098815918, 0.10627055168151855, 2.4818854331970215, 0.7961533069610596], [59, 106, 107, 486, 500], [1, 1, 1, 0, 0]]
5
<class 'pandas.core.frame.DataFrame'>
Datei da
<class 'pandas.core.frame.DataFrame'>
Habe fertig
